# 6.2. Create Lag Feature

Point of this notebook: Mana yang lebih baik, membuat feature lag dengan data 6 tahun lalu tetapi mengorbankan data row selama 6 tahun, atau apa adanya?

In [1]:
### Create Feature: Lag 6 years ago

In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv('./dataset/train_ts_feature_added.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])
df_train['source'] = 'df_train'

df_test = pd.read_csv('./dataset/test_ts_feature_added.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])
df_test['source'] = 'df_test'

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   datetime_iso     341880 non-null  datetime64[ns, UTC]
 1   temp             341880 non-null  float64            
 2   d_point          341880 non-null  float64            
 3   feels            341880 non-null  float64            
 4   min_temp         341880 non-null  float64            
 5   max_temp         341880 non-null  float64            
 6   prssr            341880 non-null  float64            
 7   hum              341880 non-null  float64            
 8   wind_spd         341880 non-null  float64            
 9   wind_deg         341880 non-null  float64            
 10  rain_1h          341880 non-null  float64            
 11  clouds           341880 non-null  float64            
 12  day_of_the_week  341880 non-null  int64              
 13 

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   datetime_iso     49368 non-null  datetime64[ns, UTC]
 1   temp             49368 non-null  float64            
 2   d_point          49368 non-null  float64            
 3   feels            49368 non-null  float64            
 4   min_temp         49368 non-null  float64            
 5   max_temp         49368 non-null  float64            
 6   prssr            49368 non-null  float64            
 7   hum              49368 non-null  float64            
 8   wind_spd         49368 non-null  float64            
 9   wind_deg         49368 non-null  float64            
 10  rain_1h          0 non-null      float64            
 11  clouds           49368 non-null  float64            
 12  day_of_the_week  49368 non-null  int64              
 13  month           

In [6]:
df = pd.concat([df_train, df_test], axis=0)  # Stacking DataFrames vertically

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391248 entries, 0 to 49367
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   datetime_iso     391248 non-null  datetime64[ns, UTC]
 1   temp             391248 non-null  float64            
 2   d_point          391248 non-null  float64            
 3   feels            391248 non-null  float64            
 4   min_temp         391248 non-null  float64            
 5   max_temp         391248 non-null  float64            
 6   prssr            391248 non-null  float64            
 7   hum              391248 non-null  float64            
 8   wind_spd         391248 non-null  float64            
 9   wind_deg         391248 non-null  float64            
 10  rain_1h          341880 non-null  float64            
 11  clouds           391248 non-null  float64            
 12  day_of_the_week  391248 non-null  int64              
 13  

In [8]:
# Specify the number of years to lag
lag_years = 6

# Calculate the lagged date
df['lag_rain_1h_6_years_date'] = df['datetime_iso'] - pd.DateOffset(years=lag_years)

df['lag_rain_1h_6_years_date'] = pd.to_datetime(df['lag_rain_1h_6_years_date'])

# Merge the DataFrame with itself to get lag_rain_1h_6_years_value
df =df.merge(df[['datetime_iso', 'rain_1h']], left_on='lag_rain_1h_6_years_date', right_on='datetime_iso', suffixes=('', '_lag'))

# Rename the 'rain_1h_lag' column to 'lag_rain_1h_6_years_value'
df.rename(columns={'rain_1h_lag': 'lag_rain_1h_6_years_value'}, inplace=True)

# Drop the duplicated datetime_iso column
df.drop(columns='datetime_iso_lag', inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 338640 entries, 0 to 338639
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   datetime_iso               338640 non-null  datetime64[ns, UTC]
 1   temp                       338640 non-null  float64            
 2   d_point                    338640 non-null  float64            
 3   feels                      338640 non-null  float64            
 4   min_temp                   338640 non-null  float64            
 5   max_temp                   338640 non-null  float64            
 6   prssr                      338640 non-null  float64            
 7   hum                        338640 non-null  float64            
 8   wind_spd                   338640 non-null  float64            
 9   wind_deg                   338640 non-null  float64            
 10  rain_1h                    289272 non-null  float64     

In [10]:
df_train = df[df['source'] == 'df_train']
df_test = df[df['source'] == 'df_test']

# Remove the 'source' column if you no longer need it
df_train = df_train.drop(columns=['source'])
df_test = df_test.drop(columns=['source','rain_1h'])

In [18]:
df_train.sample(30)

,datetime_iso,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,rain_1h,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean,lag_rain_1h_6_years_date,lag_rain_1h_6_years_value
275422,2016-06-02 22:00:00+00:00,27.16,25.38,31.20,25.48,27.76,1011.0,90.0,0.27,281.0,0.41,97.0,3,6,0,0.277009,0.315871,0.351281,2010-06-02 22:00:00+00:00,0.00
76215,1993-09-11 15:00:00+00:00,24.42,23.39,25.37,23.86,25.29,1013.0,94.0,1.74,234.0,0.00,97.0,5,9,0,0.266902,0.235011,0.257367,1987-09-11 15:00:00+00:00,0.00
56504,1991-06-13 08:00:00+00:00,30.78,25.14,37.60,29.82,32.20,1010.0,72.0,1.49,130.0,0.95,97.0,3,6,0,0.299968,0.315871,0.354803,1985-06-13 08:00:00+00:00,0.14
130542,1999-11-23 06:00:00+00:00,29.41,24.73,34.96,28.57,30.39,1007.0,76.0,1.81,164.0,0.00,94.0,1,11,1,0.477714,0.438892,0.473431,1993-11-23 06:00:00+00:00,1.96
31294,1988-07-27 22:00:00+00:00,24.57,23.72,26.65,24.03,25.27,1011.0,95.0,1.58,226.0,0.14,93.0,2,7,0,0.231902,0.268213,0.273019,1982-07-27 22:00:00+00:00,0.00
176887,2005-03-07 07:00:00+00:00,30.39,25.00,36.82,28.89,32.63,1012.0,73.0,1.11,135.0,0.42,77.0,0,3,1,0.433269,0.453459,0.464646,1999-03-07 07:00:00+00:00,0.33
171323,2004-07-18 11:00:00+00:00,27.19,25.03,26.65,26.55,28.09,1009.0,92.0,1.27,213.0,0.00,100.0,6,7,0,0.275075,0.268213,0.254351,1998-07-18 11:00:00+00:00,0.00
83755,1994-07-22 19:00:00+00:00,23.68,23.00,24.61,23.06,24.66,1011.0,96.0,1.98,226.0,0.00,97.0,4,7,0,0.271592,0.268213,0.254351,1988-07-22 19:00:00+00:00,0.00
74319,1993-06-24 15:00:00+00:00,25.31,23.92,26.30,24.45,27.92,1010.0,92.0,1.78,216.0,0.00,97.0,3,6,0,0.281229,0.315871,0.290056,1987-06-24 15:00:00+00:00,0.00
50348,1990-09-29 20:00:00+00:00,24.36,23.68,25.36,23.19,25.73,1011.0,96.0,1.46,222.0,1.00,100.0,5,9,0,0.178483,0.235011,0.224701,1984-09-29 20:00:00+00:00,0.00


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289272 entries, 0 to 289271
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   datetime_iso               289272 non-null  datetime64[ns, UTC]
 1   temp                       289272 non-null  float64            
 2   d_point                    289272 non-null  float64            
 3   feels                      289272 non-null  float64            
 4   min_temp                   289272 non-null  float64            
 5   max_temp                   289272 non-null  float64            
 6   prssr                      289272 non-null  float64            
 7   hum                        289272 non-null  float64            
 8   wind_spd                   289272 non-null  float64            
 9   wind_deg                   289272 non-null  float64            
 10  rain_1h                    289272 non-null  float64     

In [13]:
df_test.head()

,datetime_iso,temp,d_point,feels,min_temp,max_temp,prssr,hum,wind_spd,wind_deg,clouds,day_of_the_week,month,season,yearly_mean,monthly_mean,weekly_mean,lag_rain_1h_6_years_date,lag_rain_1h_6_years_value
289272,2018-01-01 00:00:00+00:00,26.59,23.66,26.59,26.02,27.16,1009.0,84.0,1.45,355.0,97.0,0,1,1,0.492286,0.442561,0.485107,2012-01-01 00:00:00+00:00,0.55
289273,2018-01-01 01:00:00+00:00,26.51,24.92,26.51,26.06,28.04,1009.0,91.0,1.67,351.0,95.0,0,1,1,0.492286,0.442561,0.485107,2012-01-01 01:00:00+00:00,0.31
289274,2018-01-01 02:00:00+00:00,28.68,25.71,34.68,28.03,29.30,1009.0,84.0,1.72,345.0,90.0,0,1,1,0.492286,0.442561,0.485107,2012-01-01 02:00:00+00:00,0.00
289275,2018-01-01 03:00:00+00:00,28.84,25.25,34.51,28.52,29.08,1008.0,81.0,1.49,339.0,91.0,0,1,1,0.492286,0.442561,0.485107,2012-01-01 03:00:00+00:00,0.00
289276,2018-01-01 04:00:00+00:00,29.75,24.62,35.38,29.31,30.57,1007.0,74.0,1.39,339.0,96.0,0,1,1,0.492286,0.442561,0.485107,2012-01-01 04:00:00+00:00,0.00


In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49368 entries, 289272 to 338639
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   datetime_iso               49368 non-null  datetime64[ns, UTC]
 1   temp                       49368 non-null  float64            
 2   d_point                    49368 non-null  float64            
 3   feels                      49368 non-null  float64            
 4   min_temp                   49368 non-null  float64            
 5   max_temp                   49368 non-null  float64            
 6   prssr                      49368 non-null  float64            
 7   hum                        49368 non-null  float64            
 8   wind_spd                   49368 non-null  float64            
 9   wind_deg                   49368 non-null  float64            
 10  clouds                     49368 non-null  float64            
 

In [15]:
df_train.to_csv('./dataset/train_ts_feature_added_2.csv', index=False)
df_test.to_csv('./dataset/test_ts_feature_added_2.csv', index=False)
